In [619]:
import pandas as pd
import http.client
import json
import csv
import os
from textblob import TextBlob
import numpy as np
import time
import gc
import random
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from datetime import datetime, timedelta
from dateutil import tz
from pytz import timezone
import pytz

In [620]:
def get_sentiment(text):
    sentDict = {}
    if text != "":
        doc = TextBlob(text)
        sentDict['polarity'] = doc.sentiment.polarity
        sentDict['subjectivity'] = doc.sentiment.subjectivity
    else:
        sentDict['polarity'] = 0
        sentDict['subjectivity'] = 0
        
    return sentDict['polarity'], sentDict['subjectivity']

In [621]:
def timeconvert(utc):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')
    utc = utc.replace(tzinfo=from_zone)
    est = utc.astimezone(to_zone)
    return est

In [622]:
post14 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2014.txt")
post15 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2015.txt")
post16 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2016.txt")
post17 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2017.txt")
post18 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2018.txt")
post19 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2019.txt")
post20 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2020.txt")
post21 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2021.txt")
frames = [post14,post15,post16,post17,post18,post19,post20,post21]
post = pd.concat(frames)

In [623]:
post['created_utc'] = pd.to_datetime(post['created_utc'])

In [624]:
post.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844931 entries, 0 to 47199
Data columns (total 10 columns):
created_utc              844931 non-null datetime64[ns]
submissionID             844931 non-null object
full_link                844931 non-null object
author                   844931 non-null object
author_flair_richtext    844931 non-null object
num_comments             844931 non-null int64
score                    844931 non-null int64
title                    844931 non-null object
selftext                 844931 non-null object
link                     844931 non-null object
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 70.9+ MB


In [625]:
post.head()

,created_utc,submissionID,full_link,author,author_flair_richtext,num_comments,score,title,selftext,link
0,2014-12-31,2qzjqr,,livebitcoinnews,,0,1,"CheapAir now Taking over Bitcoin, Dogecoin and...",,https://www.reddit.com/r/Bitcoin/comments/2qzj...
1,2014-12-31,2qziky,,BitStashCTO,,6,30,"If ever Bitcoin had a use case, Venezuela is it !",,https://www.reddit.com/r/Bitcoin/comments/2qzi...
2,2014-12-31,2qziao,,SThist,,11,134,Police suspect only 1 percent taken by Mt. Gox...,,https://www.reddit.com/r/Bitcoin/comments/2qzi...
3,2014-12-31,2qzhov,,bitsforall,,1,0,2015 - WTF - Ben Lawsky did you miss your own ...,,https://www.reddit.com/r/Bitcoin/comments/2qzh...
4,2014-12-31,2qzh5h,,bittopia,,27,33,Do you guys stash dummy BTC wallets everywhere...,I have 10 wallets on a few PCs and mobile devi...,https://www.reddit.com/r/Bitcoin/comments/2qzh...


In [626]:
post14_id = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/posts/post2014.txt")
post15_id = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/posts/post2015.txt")
post16_id = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/posts/post2016.txt")
post17_id = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/posts/post2017.txt")
post18_id = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/posts/post2018.txt")
post19_id = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/posts/post2019.txt")
post20_id = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/posts/post2020.txt")
post21_id = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/posts/post2021.txt")
frames = [post14_id,post15_id,post16_id,post17_id,post18_id,post19_id,post20_id,post21_id]
post_id = pd.concat(frames)


In [627]:
post_id['created_utc'] = pd.to_datetime(post_id['created_utc'])

In [628]:
post_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854731 entries, 0 to 56999
Data columns (total 2 columns):
created_utc     854731 non-null datetime64[ns]
submissionID    854731 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 19.6+ MB


In [629]:
post_id.head()

,created_utc,submissionID
0,2014-12-31 23:53:56,2qzfks
1,2014-12-31 23:32:44,2qze3l
2,2014-12-31 23:32:19,2qze2q
3,2014-12-31 23:14:04,2qzcro
4,2014-12-31 22:48:58,2qzb1o


In [630]:
post_id['created_est'] = post_id['created_utc'].apply(lambda x: timeconvert(x).date())

In [631]:
post_id['created_est'] = pd.to_datetime(post_id['created_est'])

In [632]:
#post['polarity'] = post.apply(get_sentiment(str(post['selftext'])), axis=1)
post['polarity'] = post['selftext'].apply(lambda x: TextBlob(str(x)).sentiment[0])
post['subjectivity'] = post['selftext'].apply(lambda x: TextBlob(str(x)).sentiment[1])
#post['subjectivity'] = post.apply(get_sentiment(str(post['selftext']))[1], axis=1)

In [633]:
post_update = post_id.merge(post, how='inner', left_on='submissionID', right_on='submissionID' )

In [634]:
post_update = post_update.drop(['created_utc_x', 'created_utc_y'], axis=1)

In [635]:
post_update = post_update.drop_duplicates()

In [636]:
post_update.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844651 entries, 0 to 845439
Data columns (total 12 columns):
submissionID             844651 non-null object
created_est              844651 non-null datetime64[ns]
full_link                844651 non-null object
author                   844651 non-null object
author_flair_richtext    844651 non-null object
num_comments             844651 non-null int64
score                    844651 non-null int64
title                    844651 non-null object
selftext                 844651 non-null object
link                     844651 non-null object
polarity                 844651 non-null float64
subjectivity             844651 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 83.8+ MB


In [637]:
post_update.head()

,submissionID,created_est,full_link,author,author_flair_richtext,num_comments,score,title,selftext,link,polarity,subjectivity
0,2qzfks,2014-12-31,,btcRaleigh,,19,57,Random Bitcoin fact I learned today,Bitcoin addresses consist of random digits and...,https://www.reddit.com/r/Bitcoin/comments/2qzf...,-0.2500,0.25
4,2qze3l,2014-12-31,,abrar12345,,21,13,Living off of Bitcoin Mining,Is it possible to earn an average Us Salary(Af...,https://www.reddit.com/r/Bitcoin/comments/2qze...,0.0125,0.65
8,2qze2q,2014-12-31,,flibbrMarketplace,,44,10,Will this be the year that bitcoin and other c...,,https://www.reddit.com/r/Bitcoin/comments/2qze...,0.0000,0.00
12,2qzcro,2014-12-31,,kingobtc,,2,2,New Years and 1K followers celebration $20 giv...,,https://www.reddit.com/r/Bitcoin/comments/2qzc...,0.0000,0.00
16,2qzb1o,2014-12-31,,cryptoreporter,,3,1,How I Got Into Bitoin - a chat log from 2011,,https://www.reddit.com/r/Bitcoin/comments/2qzb...,0.0000,0.00


In [638]:
comment14 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2014.txt")
comment15 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2015.txt")
comment16 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2016.txt")
comment17 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2017.txt")
comment18 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2018.txt")
comment19 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2019.txt")
comment20 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2020.txt")
comment21 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2021.txt")
frames = [comment14,comment15,comment16,comment17,comment18,comment19,comment20,comment21]
comment = pd.concat(frames)

/Users/jonhuaqin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [639]:
comment['created_utc'] = pd.to_datetime(comment['created_utc'])
comment["post_id"] = comment["link_id"].str.split("_").str[-1]

In [640]:
comment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2154200 entries, 0 to 62299
Data columns (total 12 columns):
author                   object
author_flair_richtext    object
body                     object
commentID                object
created_utc              datetime64[ns]
is_submitier             object
is_submitter             object
link                     object
link_id                  object
parent_id                object
score                    int64
post_id                  object
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 213.7+ MB


In [641]:
comment.head()

,author,author_flair_richtext,body,commentID,created_utc,is_submitier,is_submitter,link,link_id,parent_id,score,post_id
0,statoshi,,The direct incentive is to have your own trust...,cnaz5kk,2014-12-31,NaN,,,t3_2qygsn,t1_cnas5eg,8,2qygsn
1,throughnothing,,"Though as hardware wallets go, ledger has some...",cnaz3u1,2014-12-31,NaN,,,t3_2qxrnx,t1_cnaobhk,0,2qxrnx
2,empraptor,,I guess that's true in a sense.\n\nIt's just t...,cnaz3sc,2014-12-31,NaN,,,t3_2qx1sz,t1_cnad60o,1,2qx1sz
3,tulipfutures,,"Or you'll realize, perhaps in another year, th...",cnaz3s1,2014-12-31,NaN,,,t3_2qze2q,t1_cnayywz,-16,2qze2q
4,odal_limbs,,"For me, 2014 was the year where the idea that ...",cnaz3mm,2014-12-31,NaN,,,t3_2qz6vb,t1_cnayo12,10,2qz6vb


In [642]:
#comment = comment.iloc[0:0]

In [643]:
comment['polarity'] = comment['body'].apply(lambda x: TextBlob(str(x)).sentiment[0])
comment['subjectivity'] = comment['body'].apply(lambda x: TextBlob(str(x)).sentiment[1])


In [644]:
comment.head()

,author,author_flair_richtext,body,commentID,created_utc,is_submitier,is_submitter,link,link_id,parent_id,score,post_id,polarity,subjectivity
0,statoshi,,The direct incentive is to have your own trust...,cnaz5kk,2014-12-31,NaN,,,t3_2qygsn,t1_cnas5eg,8,2qygsn,0.237500,0.700000
1,throughnothing,,"Though as hardware wallets go, ledger has some...",cnaz3u1,2014-12-31,NaN,,,t3_2qxrnx,t1_cnaobhk,0,2qxrnx,0.300000,0.400000
2,empraptor,,I guess that's true in a sense.\n\nIt's just t...,cnaz3sc,2014-12-31,NaN,,,t3_2qx1sz,t1_cnad60o,1,2qx1sz,0.118939,0.409242
3,tulipfutures,,"Or you'll realize, perhaps in another year, th...",cnaz3s1,2014-12-31,NaN,,,t3_2qze2q,t1_cnayywz,-16,2qze2q,0.080357,0.455357
4,odal_limbs,,"For me, 2014 was the year where the idea that ...",cnaz3mm,2014-12-31,NaN,,,t3_2qz6vb,t1_cnayo12,10,2qz6vb,0.384921,0.453175


In [645]:
temp = comment.copy()
temp['polarity'] = comment['polarity'].replace(0, np.NaN)
temp['subjectivity'] = comment['subjectivity'].replace(0, np.NaN)

In [646]:
temp.head(20)

,author,author_flair_richtext,body,commentID,created_utc,is_submitier,is_submitter,link,link_id,parent_id,score,post_id,polarity,subjectivity
0,statoshi,,The direct incentive is to have your own trust...,cnaz5kk,2014-12-31,NaN,,,t3_2qygsn,t1_cnas5eg,8,2qygsn,0.237500,0.700000
1,throughnothing,,"Though as hardware wallets go, ledger has some...",cnaz3u1,2014-12-31,NaN,,,t3_2qxrnx,t1_cnaobhk,0,2qxrnx,0.300000,0.400000
2,empraptor,,I guess that's true in a sense.\n\nIt's just t...,cnaz3sc,2014-12-31,NaN,,,t3_2qx1sz,t1_cnad60o,1,2qx1sz,0.118939,0.409242
3,tulipfutures,,"Or you'll realize, perhaps in another year, th...",cnaz3s1,2014-12-31,NaN,,,t3_2qze2q,t1_cnayywz,-16,2qze2q,0.080357,0.455357
4,odal_limbs,,"For me, 2014 was the year where the idea that ...",cnaz3mm,2014-12-31,NaN,,,t3_2qz6vb,t1_cnayo12,10,2qz6vb,0.384921,0.453175
5,bigtimetimmyjim22,,If you threw dimes at people on the street wou...,cnaz35g,2014-12-31,NaN,,,t3_2qyyuj,t1_cnavgwz,1,2qyyuj,NaN,NaN
6,camraid,,You are saying that demand for premium items w...,cnaz303,2014-12-31,NaN,,,t3_2qdwmn,t1_cn5owe3,1,2qdwmn,0.072500,0.525000
7,gunslinger_006,,"Its called ""face"". Its an important concept i...",cnaz2w1,2014-12-31,NaN,,,t3_2qyyy9,t1_cnayx2w,4,2qyyy9,0.146667,0.323333
8,btcRaleigh,,Tis true! It is a bit of a gray area. I'm sure...,cnaz2fq,2014-12-31,NaN,,,t3_2qulhf,t1_cn9utha,1,2qulhf,0.468750,0.769444
9,MistakeNotDotDotDot,,It's hard to find global statistics of religio...,cnaz28k,2014-12-31,NaN,,,t3_2qwklp,t1_cnao6o5,1,2qwklp,-0.089444,0.166111


In [647]:
comment1 = temp.groupby('post_id')['author'].nunique().to_frame(name='Num_of_repliedids')
comment2 = temp.groupby('post_id').agg({'score': 'mean', 'polarity': 'mean', 'subjectivity': 'mean'})
comment3 = comment2[['score', 'polarity', 'subjectivity']].rename(columns={'score': 'comments_score', 'polarity' : 'polarity_c', 'subjectivity':'subjectivity_c'})
comment_s = comment1.merge(comment3, how='inner', left_on='post_id', right_on='post_id')

In [648]:
comment_s.head()

,Num_of_repliedids,comments_score,polarity_c,subjectivity_c
post_id,,,,
1gsd99,1,1.0,0.193750,0.400000
1im6hw,2,0.5,0.616667,0.716667
1j2voc,1,1.0,-0.200000,0.800000
1j66ry,1,1.0,0.100000,0.200000
1j7ybq,1,1.0,NaN,NaN


In [649]:
comment_s.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243006 entries, 1gsd99 to m74i9n
Data columns (total 4 columns):
Num_of_repliedids    243006 non-null int64
comments_score       243006 non-null float64
polarity_c           209121 non-null float64
subjectivity_c       214263 non-null float64
dtypes: float64(3), int64(1)
memory usage: 9.3+ MB


In [650]:
post_all = post_update.merge(comment_s, how='inner', left_on='submissionID', right_on='post_id' )

In [651]:
post_all['next_date'] = pd.DatetimeIndex(post_all.created_est) + pd.DateOffset(1)

In [652]:
post_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241832 entries, 0 to 241831
Data columns (total 17 columns):
submissionID             241832 non-null object
created_est              241832 non-null datetime64[ns]
full_link                241832 non-null object
author                   241832 non-null object
author_flair_richtext    241832 non-null object
num_comments             241832 non-null int64
score                    241832 non-null int64
title                    241832 non-null object
selftext                 241832 non-null object
link                     241832 non-null object
polarity                 241832 non-null float64
subjectivity             241832 non-null float64
Num_of_repliedids        241832 non-null int64
comments_score           241832 non-null float64
polarity_c               208202 non-null float64
subjectivity_c           213312 non-null float64
next_date                241832 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(3), object

In [653]:
post_all.head()

,submissionID,created_est,full_link,author,author_flair_richtext,num_comments,score,title,selftext,link,polarity,subjectivity,Num_of_repliedids,comments_score,polarity_c,subjectivity_c,next_date
0,2qzfks,2014-12-31,,btcRaleigh,,19,57,Random Bitcoin fact I learned today,Bitcoin addresses consist of random digits and...,https://www.reddit.com/r/Bitcoin/comments/2qzf...,-0.2500,0.25,6,1.285714,0.082197,0.477462,2015-01-01
1,2qze3l,2014-12-31,,abrar12345,,21,13,Living off of Bitcoin Mining,Is it possible to earn an average Us Salary(Af...,https://www.reddit.com/r/Bitcoin/comments/2qze...,0.0125,0.65,3,2.333333,0.241971,0.592432,2015-01-01
2,2qze2q,2014-12-31,,flibbrMarketplace,,44,10,Will this be the year that bitcoin and other c...,,https://www.reddit.com/r/Bitcoin/comments/2qze...,0.0000,0.00,7,1.333333,0.136614,0.480636,2015-01-01
3,2qzcro,2014-12-31,,kingobtc,,2,2,New Years and 1K followers celebration $20 giv...,,https://www.reddit.com/r/Bitcoin/comments/2qzc...,0.0000,0.00,1,2.000000,0.291667,0.283333,2015-01-01
4,2qzb1o,2014-12-31,,cryptoreporter,,3,1,How I Got Into Bitoin - a chat log from 2011,,https://www.reddit.com/r/Bitcoin/comments/2qzb...,0.0000,0.00,1,1.000000,-0.058183,0.404026,2015-01-01


In [654]:
post_all['polarity'] = post_all['polarity'].replace(0, np.NaN)
post_all['subjectivity'] = post_all['subjectivity'].replace(0, np.NaN)

In [655]:
post1 = post_all.groupby('next_date')['submissionID'].nunique().to_frame(name='Num_of_submissionID')
post2 = post_all.groupby('next_date')['author'].nunique().to_frame(name='Num_of_author')
post3 = post_all.groupby('next_date').agg({'num_comments': 'sum', 'Num_of_repliedids': 'sum', 'score': 'mean', 'comments_score': 'mean', 'polarity': 'mean', 'subjectivity': 'mean', 'polarity_c': 'mean', 'subjectivity_c': 'mean' })


In [656]:
post3.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2612 entries, 2014-01-01 to 2021-03-18
Data columns (total 8 columns):
num_comments         2612 non-null int64
Num_of_repliedids    2612 non-null int64
score                2612 non-null float64
comments_score       2612 non-null float64
polarity             2596 non-null float64
subjectivity         2596 non-null float64
polarity_c           2606 non-null float64
subjectivity_c       2608 non-null float64
dtypes: float64(6), int64(2)
memory usage: 183.7 KB


In [657]:
join = post1.merge(post2, how='inner', left_on='next_date', right_on='next_date')
post_s = join.merge(post3, how='inner', left_on='next_date', right_on='next_date')

In [658]:
post_s.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2612 entries, 2014-01-01 to 2021-03-18
Data columns (total 10 columns):
Num_of_submissionID    2612 non-null int64
Num_of_author          2612 non-null int64
num_comments           2612 non-null int64
Num_of_repliedids      2612 non-null int64
score                  2612 non-null float64
comments_score         2612 non-null float64
polarity               2596 non-null float64
subjectivity           2596 non-null float64
polarity_c             2606 non-null float64
subjectivity_c         2608 non-null float64
dtypes: float64(6), int64(4)
memory usage: 224.5 KB


In [659]:
post_s.head()

,Num_of_submissionID,Num_of_author,num_comments,Num_of_repliedids,score,comments_score,polarity,subjectivity,polarity_c,subjectivity_c
next_date,,,,,,,,,,
2014-01-01,35,26,1158,120,50.514286,1.962018,0.170922,0.452008,0.133471,0.499066
2014-01-02,100,83,2602,521,54.640000,1.951958,0.150230,0.469588,0.128377,0.511687
2014-01-03,105,82,4353,531,60.638095,1.636724,0.084931,0.504696,0.158921,0.530261
2014-01-04,117,100,3758,488,54.205128,1.928624,0.138263,0.480636,0.153357,0.562601
2014-01-05,130,105,3027,577,42.107692,1.914569,0.125373,0.502517,0.142905,0.523303


In [660]:
bitcoin = pd.read_csv('BTC-USD.csv')

In [661]:
bitcoin['delta'] = bitcoin['Adj Close'] - bitcoin['Open']
bitcoin['increase'] = bitcoin.apply(lambda x: 1 if x['delta']>0 else 0, axis=1)
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])


In [662]:
bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2373 entries, 0 to 2372
Data columns (total 9 columns):
Date         2373 non-null datetime64[ns]
Open         2369 non-null float64
High         2369 non-null float64
Low          2369 non-null float64
Close        2369 non-null float64
Adj Close    2369 non-null float64
Volume       2369 non-null float64
delta        2369 non-null float64
increase     2373 non-null int64
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 167.0 KB


In [663]:
bitcoin.head()

,Date,Open,High,Low,Close,Adj Close,Volume,delta,increase
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0,-8.529999,0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0,-32.419983,0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0,-29.307007,0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0,14.230988,1
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0,-9.263977,0


In [664]:
dataset_temp = post_s.merge(bitcoin, how='inner', left_on='next_date', right_on='Date')
dataset_temp = dataset_temp.replace(np.NaN, 0)

dataset = dataset_temp.drop(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'delta' ],axis=1)


In [665]:
dataset1 = dataset.round({'score': 2, 'comments_score': 2, 'polarity':2, 'subjectivity':2, 'polarity_c':2, 'subjectivity_c':2 })

In [666]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2351 entries, 0 to 2350
Data columns (total 12 columns):
Num_of_submissionID    2351 non-null int64
Num_of_author          2351 non-null int64
num_comments           2351 non-null int64
Num_of_repliedids      2351 non-null int64
score                  2351 non-null float64
comments_score         2351 non-null float64
polarity               2351 non-null float64
subjectivity           2351 non-null float64
polarity_c             2351 non-null float64
subjectivity_c         2351 non-null float64
Date                   2351 non-null datetime64[ns]
increase               2351 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(5)
memory usage: 238.8 KB


In [667]:
dataset1.head()

,Num_of_submissionID,Num_of_author,num_comments,Num_of_repliedids,score,comments_score,polarity,subjectivity,polarity_c,subjectivity_c,Date,increase
0,102,85,2587,598,44.40,1.92,0.10,0.48,0.14,0.55,2014-09-17,0
1,166,126,3070,903,34.98,1.60,0.12,0.48,0.17,0.54,2014-09-18,0
2,82,70,2606,338,60.50,1.30,0.10,0.50,0.17,0.53,2014-09-19,0
3,118,94,3320,620,40.34,1.70,0.12,0.44,0.13,0.53,2014-09-20,1
4,85,68,2370,579,42.69,2.11,0.12,0.50,0.15,0.53,2014-09-21,0


In [668]:
x_data = dataset1.loc[:, dataset1.columns != "increase"]
y_data = dataset1.loc[:, "increase"]

In [669]:
x_train_t, x_test_t, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, shuffle = True, random_state = 614)

In [670]:
x_train = x_train_t.loc[:, x_train_t.columns != "Date"]
x_test = x_test_t.loc[:, x_test_t.columns != "Date"]
x_train_date = x_train_t.loc[:, "Date"]
x_test_date = x_test_t.loc[:, "Date"]


In [671]:
x_train_date.head()

1344   2018-06-08
717    2016-09-03
2102   2020-07-05
735    2016-09-21
15     2014-10-02
Name: Date, dtype: datetime64[ns]

In [672]:
#export
class LinearRegressionModel():
    
    # points [2]
    def linearClassifier(self,x_train, x_test, y_train):
        # TODO: Create a LinearRegression classifier and train it.
        # args: pandas dataframe, pandas dataframe, pandas series
        # return: numpy array, numpy array
        # -------------------------------
        # ADD CODE HERE
        linear = LinearRegression().fit(x_train,y_train)
        y_predict_train = linear.predict(x_train)
        y_predict_test = linear.predict(x_test)
        
        # -------------------------------
        return y_predict_train, y_predict_test

    # points [1]
    def lgTrainAccuracy(self,y_train,y_predict_train):
        # TODO: Return accuracy (on the training set) using the accuracy_score method.
        # Note: Round the output values greater than or equal to 0.5 to 1 and those less than 0.5 to 0. You can use any method that satisfies the requriements.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE 
        #y_predict_train_round = [round(k) for k in y_predict_train]
        for i in range(len(y_predict_train)):
            if y_predict_train[i] >= 0.5: y_predict_train[i] = 1
            else: y_predict_train[i] = 0
        train_accuracy = accuracy_score(y_train, y_predict_train)
 
        # -------------------------------   
        return train_accuracy
    
    # points [1]
    def lgTestAccuracy(self,y_test,y_predict_test):
        # TODO: Return accuracy (on the testing set) using the accuracy_score method.
        # Note: Round the output values greater than or equal to 0.5 to 1 and those less than 0.5 to 0. You can use any method that satisfies the requriements.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        #y_predict_test_round = [round(k) for k in y_predict_test]
        for i in range(len(y_predict_test)):
            if y_predict_test[i] >= 0.5: y_predict_test[i] = 1
            else: y_predict_test[i] = 0
        test_accuracy = accuracy_score(y_test, y_predict_test)
        # -------------------------------
        return test_accuracy
    


In [673]:
Linear = LinearRegressionModel()


In [674]:
y_predict_train, y_predict_test = Linear.linearClassifier(x_train, x_test, y_train)

In [675]:
train_accuracy = Linear.lgTrainAccuracy(y_train,y_predict_train)

In [676]:
test_accuracy = Linear.lgTestAccuracy(y_test,y_predict_test)

In [677]:
print("train_accuracy = " + str(train_accuracy))
print("test_accuracy = " + str(test_accuracy))

train_accuracy = 0.5495440729483283
test_accuracy = 0.5354107648725213


In [678]:
linear_model = {'x_test_date': x_test_date, 'y_predict_test': y_predict_test, 'y_test': y_test }
linear_model_result = pd.DataFrame(linear_model)

In [679]:
linear_model_result.to_csv('linear_model_result.csv')

In [680]:
#export
class RFClassifier():
    
    # points [2]
    def randomForestClassifier(self,x_train,x_test, y_train):
        # TODO: Create a RandomForestClassifier and train it. Set Random state to 614.
        # args: pandas dataframe, pandas dataframe, pandas series
        # return: RandomForestClassifier object, numpy array, numpy array
        # -------------------------------
        # ADD CODE HERE
        rf_clf = RandomForestClassifier(random_state = 614).fit(x_train, y_train)
        y_predict_train = rf_clf.predict(x_train)
        y_predict_test = rf_clf.predict(x_test)
        # -------------------------------
        return rf_clf,y_predict_train, y_predict_test
    
    # points [1]
    def rfTrainAccuracy(self,y_train,y_predict_train):
        # TODO: Return accuracy on the training set using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        train_accuracy = accuracy_score(y_train, y_predict_train)
        # -------------------------------
        return train_accuracy
    
    # points [1]
    def rfTestAccuracy(self,y_test,y_predict_test):
        # TODO: Return accuracy on the test set using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        test_accuracy = accuracy_score(y_test, y_predict_test)
        # -------------------------------
        return test_accuracy
    
# Q3.3.1 Feature Importance
    
    # points [1]
    def rfFeatureImportance(self,rf_clf):
        # TODO: Determine the feature importance as evaluated by the Random Forest Classifier.
        # args: RandomForestClassifier object
        # return: float array
        # -------------------------------
        # ADD CODE HERE
        
        feature_importance = rf_clf.feature_importances_
        # -------------------------------
        return feature_importance
    
    # points [1]
    def sortedRFFeatureImportanceIndicies(self,rf_clf):
        # TODO: Sort them in the ascending order and return the feature numbers[0 to ...].
        #       Hint: There is a direct function available in sklearn to achieve this. Also checkout argsort() function in Python.
        # args: RandomForestClassifier object
        # return: int array
        # -------------------------------
        # ADD CODE HERE
        sorted_indices = np.argsort(rf_clf.feature_importances_)[::-1]
        # -------------------------------
        return sorted_indices
    
# Q3.3.2 Hyper-parameter Tuning

    # points [2]
    def hyperParameterTuning(self,rf_clf,x_train,y_train):
        # TODO: Tune the hyper-parameters 'n_estimators' and 'max_depth'.
        # args: RandomForestClassifier object, pandas dataframe, pandas series
        # return: GridSearchCV object
        # 'n_estimators': [4, 16, 256]
        # 'max_depth': [2, 8, 16]
        # -------------------------------
        # ADD CODE HERE
        param_grid = {'n_estimators': [4, 16, 256], 'max_depth': [2, 8, 16]}
        gscv_rfc = GridSearchCV(estimator = rf_clf, param_grid = param_grid)
        gscv_rfc.fit(x_train, y_train)
        # -------------------------------
        return gscv_rfc
    
    # points [1]
    def bestParams(self,gscv_rfc):
        # TODO: Get the best params, using .best_params_
        # args:  GridSearchCV object
        # return: parameter dict
        # -------------------------------
        # ADD CODE HERE
        best_params = gscv_rfc.best_params_
        # -------------------------------
        return best_params
    
    # points [1]
    def bestScore(self,gscv_rfc):
        # TODO: Get the best score, using .best_score_.
        # args: GridSearchCV object
        # return: float
        # -------------------------------
        # ADD CODE HERE
        best_score = gscv_rfc.best_score_
        # -------------------------------
        return best_score
    
    def randomForestClassifier_n(self,x_train,x_test, y_train):
        # TODO: Create a RandomForestClassifier and train it. Set Random state to 614.
        # args: pandas dataframe, pandas dataframe, pandas series
        # return: RandomForestClassifier object, numpy array, numpy array
        # -------------------------------
        # ADD CODE HERE
        rf_clf_n = RandomForestClassifier(random_state = 614, n_estimators = 256, max_depth = 2).fit(x_train, y_train)
        y_predict_train_n = rf_clf_n.predict(x_train)
        y_predict_test_n = rf_clf_n.predict(x_test)
        # -------------------------------
        return rf_clf_n,y_predict_train_n, y_predict_test_n
    

In [681]:
rm = RFClassifier()

In [682]:
rf_clf,y_predict_train, y_predict_test = rm.randomForestClassifier(x_train,x_test, y_train)

/Users/jonhuaqin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [683]:
train_accuracy = rm.rfTrainAccuracy(y_train,y_predict_train)

In [684]:
train_accuracy

0.9854103343465046

In [685]:
test_accuracy = rm.rfTestAccuracy(y_test,y_predict_test)

In [686]:
test_accuracy

0.4985835694050991

In [687]:
rm_model = {'x_test_date': x_test_date, 'y_predict_test': y_predict_test, 'y_test': y_test }
rm_model_result = pd.DataFrame(rm_model)

In [688]:
rm_model_result.to_csv('rm_model_result.csv')

In [689]:
gscv_rfc = rm.hyperParameterTuning(rf_clf,x_train,y_train)

/Users/jonhuaqin/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [690]:
best_params = gscv_rfc.best_params_

In [691]:
best_params

{'max_depth': 2, 'n_estimators': 256}

In [692]:
best_score = gscv_rfc.best_score_

In [693]:
best_score

0.5422492401215806

In [694]:
gscv_rfc

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False, random_state=614,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=

In [695]:
rf_clf_n,y_predict_train, y_predict_test_n = rm.randomForestClassifier_n(x_train,x_test, y_train)


In [696]:
test_accuracy = rm.rfTestAccuracy(y_test,y_predict_test_n)

In [697]:
test_accuracy

0.5325779036827195

In [698]:
rmcv_model = {'x_test_date': x_test_date, 'y_predict_test': y_predict_test_n, 'y_test': y_test }
rmcv_model_result = pd.DataFrame(rmcv_model)

In [699]:
rmcv_model_result.to_csv('rmcv_model_result.csv')

In [700]:
#export
class SupportVectorMachine():
    
# Q3.4.1 Pre-process

    # points [1]
    def dataPreProcess(self,x_train,x_test):
        # TODO: Pre-process the data to standardize it, otherwise the grid search will take much longer.
        # args: pandas dataframe, pandas dataframe
        # return: pandas dataframe, pandas dataframe
        # -------------------------------
        # ADD CODE HERE
        scaler = StandardScaler()
        scaler.fit(x_train)
        scaled_x_train = scaler.transform(x_train)
        scaled_x_test = scaler.transform(x_test)
        # -------------------------------
        return scaled_x_train, scaled_x_test
    
# Q3.4.2 Classification

    # points [1]
    def SVCClassifier(self,scaled_x_train,scaled_x_test, y_train):
        # TODO: Create a SVC classifier and train it. Set gamma = 'auto'
        # args: pandas dataframe, pandas dataframe, pandas series
        # return: numpy array, numpy array
        # -------------------------------
        # ADD CODE HERE
        svm = SVC(gamma = 'auto')
        svm.fit(scaled_x_train, y_train)
        y_predict_train = svm.predict(scaled_x_train)
        y_predict_test = svm.predict(scaled_x_test)
        # -------------------------------
        return y_predict_train,y_predict_test
    
    # points [1]
    def SVCTrainAccuracy(self,y_train,y_predict_train):
        # TODO: Return accuracy on the training set using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float 
        # -------------------------------
        # ADD CODE HERE
        y_train_pred_round = [round(k) for k in y_predict_train]
        train_accuracy = accuracy_score(y_train_pred_round, y_train)
        # -------------------------------
        return train_accuracy
    
    # points [1]
    def SVCTestAccuracy(self,y_test,y_predict_test):
        # TODO: Return accuracy on the test set using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float 
        # -------------------------------
        # ADD CODE HERE
        y_test_pred_round = [round(k) for k in y_predict_test]
        test_accuracy = accuracy_score(y_test_pred_round, y_test)        
        # -------------------------------
        return test_accuracy
    
# Q3.4.3 Hyper-parameter Tuning
    
    # points [1]
    def SVMBestScore(self, scaled_x_train, y_train):
        # TODO: Tune the hyper-parameters 'C' and 'kernel' (use rbf and linear).
        # Note: Set n_jobs = -1 and , return_train_score=Ture and gamma = 'auto'
        # args: pandas dataframe, pandas series
        # return: GridSearchCV object, float
        # -------------------------------
        svm_parameters = {'kernel':('linear', 'rbf'), 'C':[0.01, 0.1, 1.0]}
        # ADD CODE HERE
        svm = SVC(gamma = 'auto')
        #svm.fit(scaled_x_train, y_train)
        svm_cv = GridSearchCV(estimator = svm, param_grid = svm_parameters, n_jobs = -1, return_train_score=True)
        svm_cv.fit(scaled_x_train, y_train)
        best_score = svm_cv.best_score_
        # -------------------------------
        
        return svm_cv, best_score
    
    # points [1]
    def SVCClassifierParam(self,svm_cv,scaled_x_train,scaled_x_test,y_train):
        # TODO: Calculate the training and test set accuracy values after hyperparameter tuning and standardization. 
        # args: GridSearchCV object, pandas dataframe, pandas dataframe, pandas series
        # return: numpy series, numpy series
        # -------------------------------
        # ADD CODE HERE
        #svm_cv, best_score = SVMBestScore(scaled_x_train, y_train)
        y_predict_train = svm_cv.predict(scaled_x_train)
        y_predict_test = svm_cv.predict(scaled_x_test)
        # -------------------------------
        return y_predict_train, y_predict_test

    # points [1]
    def svcTrainAccuracy(self,y_train,y_predict_train):
        # TODO: Return accuracy (on the training set) using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        train_accuracy = accuracy_score(y_predict_train, y_train)
        # -------------------------------
        return train_accuracy

    # points [1]
    def svcTestAccuracy(self,y_test,y_predict_test):
        # TODO: Return accuracy (on the test set) using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        test_accuracy = accuracy_score(y_predict_test, y_test)
        # -------------------------------
        return test_accuracy
    
# Q3.4.4 Cross Validation Results

    # points [1]
    def SVMRankTestScore(self,svm_cv):
        # TODO: Return the rank test score for all hyperparameter values that you obtained in Q3.4.3. The 
        # GridSearchCV class holds a 'cv_results_' dictionary that should help you report these metrics easily.
        # args: GridSearchCV object 
        # return: int array
        # -------------------------------
        # ADD CODE HERE
        """for i in range(len(svm_cv.cv_results_["params"])):
            if svm_cv.cv_results_["params"][i] == svm_cv.best_params_:
                rank_test_score = svm_cv.cv_results_["rank_test_score"][i]
        """
        rank_test_score = svm_cv.cv_results_["rank_test_score"]
        # -------------------------------
        return rank_test_score
    
    # points [1]
    def SVMMeanTestScore(self,svm_cv):
        # TODO: Return mean test score for all of hyperparameter values that you obtained in Q3.4.3. The 
        # GridSearchCV class holds a 'cv_results_' dictionary that should help you report these metrics easily.
        # args: GridSearchCV object
        # return: float array
        # -------------------------------
        # ADD CODE HERE
        """for i in range(len(svm_cv.cv_results_["params"])):
            if svm_cv.cv_results_["params"][i] == svm_cv.best_params_:
                mean_test_score = svm_cv.cv_results_["mean_test_score"][i]
        """
        mean_test_score = svm_cv.cv_results_["mean_test_score"]
        # -------------------------------
        return mean_test_score

In [701]:
svm = SupportVectorMachine()

In [702]:
scaled_x_train, scaled_x_test = svm.dataPreProcess(x_train,x_test)
y_predict_train,y_predict_test = svm.SVCClassifier(scaled_x_train,scaled_x_test, y_train)
train_accuracy = svm.SVCTrainAccuracy(y_train,y_predict_train)
test_accuracy = svm.SVCTestAccuracy(y_test,y_predict_test)
 


In [703]:
svm_model = {'x_test_date': x_test_date, 'y_predict_test': y_predict_test, 'y_test': y_test }
svm_model_result = pd.DataFrame(svm_model)

In [704]:
svm_model_result.to_csv('svm_model_result.csv')

In [705]:
print("train_accuracy = " + str(train_accuracy))
print("test_accuracy = " + str(test_accuracy))

train_accuracy = 0.5835866261398176
test_accuracy = 0.5311614730878187


In [706]:
svm_cv, best_score = svm.SVMBestScore(scaled_x_train, y_train)
y_predict_train, y_predict_test = svm.SVCClassifierParam(svm_cv,scaled_x_train,scaled_x_test,y_train)
train_accuracy = svm.svcTrainAccuracy(y_train,y_predict_train)
test_accuracy = svm.svcTestAccuracy(y_test,y_predict_test)

/Users/jonhuaqin/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [707]:
svmcv_model = {'x_test_date': x_test_date, 'y_predict_test': y_predict_test, 'y_test': y_test }
svmcv_model_result = pd.DataFrame(svmcv_model)

In [708]:
svmcv_model_result.to_csv('svmcv_model_result.csv')

In [709]:
print("train_accuracy = " + str(train_accuracy))
print("test_accuracy = " + str(test_accuracy))

train_accuracy = 0.5519756838905775
test_accuracy = 0.5297450424929179


In [710]:
rank_test_score = svm_cv.cv_results_["rank_test_score"]
mean_test_score = svm_cv.cv_results_["mean_test_score"]


In [711]:
print(rank_test_score)


[1 1 1 1 1 6]


In [712]:
print(mean_test_score)

[0.55197568 0.55197568 0.55197568 0.55197568 0.55197568 0.54468085]


In [713]:
knn = KNeighborsClassifier().fit(x_train,y_train)
y_predict_train = knn.predict(x_train)
y_predict_test = knn.predict(x_test)

In [714]:
knn_model = {'x_test_date': x_test_date, 'y_predict_test': y_predict_test, 'y_test': y_test }
knn_model_result = pd.DataFrame(knn_model)

In [715]:
knn_model_result.to_csv('knn_model.csv')
#df.to_csv(file_name, sep='\t')

In [716]:
train_accuracy = accuracy_score(y_train, y_predict_train)
test_accuracy = accuracy_score(y_test, y_predict_test)

In [717]:
train_accuracy

0.6826747720364742

In [718]:
test_accuracy

0.509915014164306